<a href="https://colab.research.google.com/github/suleman1412/kaggle-diaries/blob/main/real_estate_prices_dutch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
raw = pd.read_csv('/kaggle/input/funda-df/funda.csv')

In [ ]:
df = raw.copy()
df

,Unnamed: 0,url,post_code,neighborhood,city,province,asking_price,living_area,asking_per_m2,longitude,latitude,house_type,plot_area,bedrooms,total_rooms,energy_label,building_form,commerical_space,roof_terrace,garden,balcony,solar_panels,broker_association,broker,broker_webpage,google_maps_url,asking_price_range,market_status,brochure_link,description,build_year
0,0,https://www.funda.nl/detail/koop/amsterdam/app...,1011vp,Rapenburg,amsterdam,Noord-Holland,385000,51,"€ 7,549",4.908872,52.370613,appartement,NaN,1,2,a,bestaandebouw,False,False,False,False,False,na,na,na,https://www.google.nl/maps/place/Nieuwegrachtj...,375000,verkocht,https://cloud.funda.nl/valentina_media/183/942...,*ENGLISH BELOW*\r\n\r\nRuim tweekamerapparteme...,1981
1,1,https://www.funda.nl/detail/koop/amsterdam/hui...,1011jn,Zuiderkerkbuurt,amsterdam,Noord-Holland,4250000,366,"€ 11,612",4.897802,52.367764,woonhuis,NaN,7,11,na,bestaandebouw,False,False,False,True,False,na,na,na,https://www.google.nl/maps/place/Staalkade+4+1...,2000000,beschikbaar,na,Op één van de mooiste plekken in het hart van ...,1733
2,2,https://www.funda.nl/detail/koop/amsterdam/app...,1011hd,Zuiderkerkbuurt,amsterdam,Noord-Holland,474000,62,"€ 7,645",4.900134,52.370686,appartement,NaN,2,3,na,bestaandebouw,False,False,False,False,False,na,na,na,https://www.google.nl/maps/place/Nieuwe+Hoogst...,450000,verkocht,na,*English text below*\r\n\r\nOp een geweldige l...,1900
3,3,https://www.funda.nl/detail/koop/amsterdam/app...,1011bh,Scheepvaarthuisbuurt,amsterdam,Noord-Holland,2500000,447,"€ 5,593",4.904404,52.373670,appartement,NaN,6,12,na,bestaandebouw,False,True,False,False,False,na,na,na,https://www.google.nl/maps/place/Binnenkant+21...,2000000,verkocht,na,Binnenkant 21 beletage en sous \r\n1011 BH Ams...,vo
4,4,https://www.funda.nl/detail/koop/amsterdam/app...,1011rz,Valkenburg,amsterdam,Noord-Holland,3500000,514,"€ 6,809",4.907461,52.367966,appartement,NaN,6,12,na,bestaandebouw,False,False,True,False,False,na,na,na,https://www.google.nl/maps/place/Nieuwe+Hereng...,2000000,verkocht,na,Op een prachtige locatie staat dit bijzondere ...,1685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253943,253943,https://www.funda.nl/detail/koop/haarlem/huis-...,2014eb,Natuurkundigenbuurt-west,haarlem,Noord-Holland,800000,143,5594,4.610116,52.369923,woonhuis,153 m²,6,7,b,bestaandebouw,False,False,True,False,False,na,na,na,https://www.google.nl/maps/place/Ohmstraat+7+2...,800000,beschikbaar,https://cloud.funda.nl/valentina_media/193/642...,"PUUR* wonen in dit heerlijke gezinshuis, grote...",1931
253944,253944,https://www.funda.nl/detail/koop/nuenen/huis-r...,5672te,Nuenen-Zuid,nuenen,Noord-Brabant,490000,130,3769,5.554063,51.465855,woonhuis,226 m²,3,4,c,bestaandebouw,False,False,True,False,False,na,na,na,https://www.google.nl/maps/place/Refelingse+Er...,450000,beschikbaar,https://cloud.funda.nl/valentina_media/193/642...,Zeer luxe geschakeld woonhuis met een inpandig...,1972
253945,253945,https://www.funda.nl/detail/koop/breda/huis-wi...,4834ph,Ypelaar,breda,Noord-Brabant,450000,130,3462,4.811380,51.570446,woonhuis,228 m²,4,5,c,bestaandebouw,False,True,True,False,False,na,na,na,https://www.google.nl/maps/place/Wildenborgstr...,450000,beschikbaar,https://cloud.funda.nl/valentina_media/193/637...,Deze keurige en uitgebouwde eindwoning ligt in...,1968
253946,253946,https://www.funda.nl/detail/koop/nootdorp/appa...,2631cr,Nootdorp CentrumWest,nootdorp,Zuid-Holland,292500,59,4958,4.394028,52.043083,appartement,NaN,na,1,a,bestaandebouw,False,False,False,False,False,na,na,na,https://www.google.nl/maps/place/Dorpsstraat+1...,275000,beschikbaar,https://cloud.funda.nl/valentina_media/193/641...,Bent u op zoek naar een aparte woning in het k...,2009


In [ ]:
# Drop outliers
outliers = [49726, 143877, 48856, 31356, 98594, 147996]
df.drop(index=outliers, inplace=True)

In [ ]:
def clean_housing_data(df):

    # Handle 'bedrooms' column
    df = df[df['bedrooms'] != 'na']
    df['bedrooms'] = df['bedrooms'].astype(int)

    # Handle 'energy_label' column
    df = df[df['energy_label'] != 'na']

    # Clean and convert 'plot_area' column
    df['plot_area'] = df['plot_area'].str.replace(' m²', '').str.replace('.', '').astype(float)
    df = df[df['plot_area'].notnull()] # This removes all apartments

    # Handle 'build_year' column
    df = df[df['build_year'].notnull()]
    df = df[~df['build_year'].isin(['vo', 'na'])]


    # Adding an age column
    df['build_year'] = df['build_year'].astype(int)
    df['age'] = 2024 - df['build_year']

    # Select relevant columns
    columns_to_keep = ['neighborhood', 'city', 'province', 'asking_price',
                       'living_area', 'longitude', 'latitude', 'plot_area',
                       'bedrooms', 'total_rooms', 'energy_label', 'building_form',
                       'commerical_space', 'roof_terrace', 'garden', 'balcony', 'solar_panels',
                       'market_status', 'age']
    df = df[columns_to_keep]


    # Clean string columns
    for col in df.select_dtypes('object').columns:
        df[col] = df[col].str.replace('-', '_').str.replace(' ', '_').str.lower()

    df.drop_duplicates(inplace=True)
    df = df[~df[['latitude','longitude']].duplicated()]
    df = df.reset_index(drop=True)
    return df

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

def encode_housing_features(df):
    # Create a copy of the dataframe to avoid modifying the original
    encoded_df = df.copy()

    # Encode boolean columns
    for col in encoded_df.select_dtypes('bool').columns:
        encoded_df[col] = encoded_df[col].astype(int)

    # Encode market_status
    le = LabelEncoder()
    encoded_df['market_status_encoded'] = le.fit_transform(encoded_df['market_status'])

    # Encode building_form
    encoded_df['building_form_encoded'] = encoded_df['building_form'].apply(lambda x: 1 if x == 'bestaandebouw' else 0)

    # Encode energy_label
    categories = [['g', 'f', 'e', 'd', 'c', 'b', 'a']]
    oe = OrdinalEncoder(categories=categories)
    encoded_df['energy_label_encoded'] = oe.fit_transform(encoded_df[['energy_label']])

    # Encode neighborhood, city, province
        # Calculate average asking prices of neighborhoods grouped by city and province
    avg_prices_sorted = encoded_df.groupby(['province', 'city', 'neighborhood'])['asking_price'].mean().reset_index().sort_values('asking_price', ascending=False)

    location_enc = OrdinalEncoder()
    location_enc.fit(avg_prices_sorted[['province', 'city', 'neighborhood']])
    encoded_copy = location_enc.transform(encoded_df[['province', 'city', 'neighborhood']])
    encoded_copy_df = pd.DataFrame(encoded_copy, columns=['province_enc', 'city_enc', 'neighborhood_enc'])

    columns_to_keep = ['province_enc', 'city_enc', 'neighborhood_enc', 'living_area', 'longitude', 'latitude', 'plot_area',
                   'bedrooms', 'total_rooms', 'commerical_space', 'roof_terrace', 'garden', 'balcony', 'solar_panels','age' ,
                    'market_status_encoded', 'building_form_encoded', 'energy_label_encoded', 'asking_price']

    encoded_df = encoded_df.join(encoded_copy_df)

    return encoded_df[columns_to_keep]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# First, we need to wrap your functions in custom transformer classes
class DataCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return clean_housing_data(X)

class FeatureEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return encode_housing_features(X)

# Now we can create the pipeline
housing_pipeline = Pipeline([
    ('cleaner', DataCleaner()),
    ('encoder', FeatureEncoder())
])

In [ ]:
processed_df = housing_pipeline.fit_transform(df)
processed_df

In [ ]:
X = processed_df.iloc[:,:-1]
y = processed_df.iloc[:,-1:].values.ravel()

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
param_grid = {
    'n_estimators': [10, 25, 50, 100],
    'max_depth': [None, 2, 4, 6, 8, 10],
    'min_samples_split': [1, 2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2']
}


# Initialize the RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=42)

# Define a scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Initialize GridSearchCV with the RandomForestRegressor and parameter grid
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=2, verbose=3, scoring=mse_scorer, n_jobs=-1)

# Fit the model on the training data
grid_search.fit(X_train, y_train)



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer, r2_score
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)

# Print out evaluation metrics
mse = mean_squared_error(y_train, y_train_pred)
mae = mean_absolute_error(y_train, y_train_pred)
rmse = mse ** 0.5
r2 = r2_score(y_train, y_train_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f'R2 score with training predictions: {r2}')
plt.hist(y_train, color='blue', alpha=0.5, label='Actual')
plt.hist(y_train_pred, color='red', alpha=0.5, label='Predicted')
plt.title('Training Data vs Training Predictions')
plt.legend()
plt.show()

In [ ]:
y_test_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_test_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f'R2 score with testing predictions: {r2}')
plt.hist(y_test, color='blue', alpha=0.5, label='Actual')
plt.hist(y_test_pred, color='red', alpha=0.5, label='Predicted')
plt.title('Testing Data vs Testing Predictions')
plt.legend()
plt.show()

In [ ]:
import joblib

In [ ]:
loaded_preprocessing_pipeline = joblib.load('best_model_randomforest.joblib')
loaded_preprocessing_pipeline

RandomForestRegressor(max_features='sqrt', min_samples_split=4, random_state=42)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Assuming you have your preprocessed data in 'processed_df'
# and your target variable is 'target'


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Set up the LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Create the LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Train the model
model = lgb.train(params, train_data, num_boost_round=1000,
                  valid_sets=[test_data])

# Make predictions
y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)

# Evaluate the model
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, y_train_pred)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

In [ ]:
y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
# Evaluate the model
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

In [ ]:
dummy_data = pd.DataFrame([{
    'province_enc': 0,
    'city_enc': 54,
    'neighborhood_enc': 2134,
    'living_area': 64,
    'longitude': 4.407900490993913,
    'latitude': 51.91541688552802,
    'plot_area': 74,
    'bedrooms': 1,
    'total_rooms': 2,
    'commerical_space': 0,
    'roof_terrace': 0,
    'garden': 0,
    'balcony': 0,
    'solar_panels': 0,
    'age': 91,
    'market_status_encoded': 1,
    'building_form_encoded': 0,
    'energy_label_encoded': 6,

}])

In [ ]:
loaded_preprocessing_pipeline.predict(dummy_data)

array([276321.44047619])